In [3]:
import LOTlib3
from LOTlib3.Grammar import Grammar
from LOTlib3.Eval import primitive
from LOTlib3.Hypotheses.LOTHypothesis import LOTHypothesis
from LOTlib3.Hypotheses.Likelihoods.BinaryLikelihood import BinaryLikelihood
from LOTlib3.Miscellaneous import q
from LOTlib3.DataAndObjects import FunctionData
from LOTlib3.Samplers.MetropolisHastings import MetropolisHastingsSampler
from LOTlib3.TopN import TopN

import numpy as np
from functools import cache
import builtins
from copy import copy
from itertools import product
from pprint import pprint

Define a grammar:

In [4]:
grammar = Grammar(start='ACTION')

grammar.add_rule('ACTION', '0', None, 3)
grammar.add_rule('ACTION', '1', None, 3)

grammar.add_rule('UTILITY', 'u_a', None, 3)
grammar.add_rule('UTILITY', 'u_b', None, 3)

grammar.add_rule('ACTION', '(%s if %s else %s)', ['ACTION', 'BOOL', 'ACTION'], 0.5)
grammar.add_rule('BOOL', '(%s == %s)', ['UTILITY', 'UTILITY'], 0.5)

# grammar.add_rule('BOOL', '(%s and %s)', ['BOOL', 'BOOL'], 1)

grammar.add_rule('UTILITY', 'max([%s, %s])', ['UTILITY', 'UTILITY'], 1)

UTILITY -> max([%s, %s])['UTILITY', 'UTILITY']	w/ p=1.0

Example of generating a sentence from the grammar:

In [28]:
grammar.generate()

(1 if (max([max([u_a, u_b]), max([u_a, u_a])]) == u_b) else 1)

Defining a hypothesis:

In [29]:
class MyHypothesis(LOTHypothesis):

    def __init__(self, **kwargs):
        LOTHypothesis.__init__(
            self, 
            grammar=grammar,
            display='lambda u_a, u_b: %s',
            **kwargs
        )

    def compute_single_likelihood(self, datum):

        """
        datum: list[float]
            For each action, its utility
        """

        action = self(*datum.input)
        utility = datum.input[action]
        return np.log(utility)

In [30]:
# Define the data
data = (
    [FunctionData(input=[0.1, 0.9], output=None)]*10 + 
    [FunctionData(input=[0.9, 0.1], output=None)]*10
)

In [ ]:
# Fit the model to the data
tn = TopN(N=10)
h0 = MyHypothesis()
for h in MetropolisHastingsSampler(h0, data, steps=1e6):
    # print(h)
    tn.add(h)

In [35]:
# Print the best found hypotheses
for h in tn.get_all(sorted=True):
    print(h.posterior_score, h.likelihood, h.prior, h)

-13.499551186744467 -2.1072103131565263 -11.39234087358794 lambda u_a, u_b: (0 if (max([u_a, u_a]) == max([u_a, u_b])) else 1)
-13.499551186744467 -2.1072103131565263 -11.39234087358794 lambda u_a, u_b: (0 if (max([max([u_b, u_b]), u_a]) == u_a) else 1)
-10.70634317730195 -2.1072103131565263 -8.599132864145423 lambda u_a, u_b: (0 if (u_a == max([u_a, u_b])) else 1)
-10.70634317730195 -2.1072103131565263 -8.599132864145423 lambda u_a, u_b: (0 if (max([u_b, u_a]) == u_a) else 1)
-10.70634317730195 -2.1072103131565263 -8.599132864145423 lambda u_a, u_b: (1 if (max([u_b, u_a]) == u_b) else 0)
-10.70634317730195 -2.1072103131565263 -8.599132864145423 lambda u_a, u_b: (0 if (u_a == max([u_b, u_a])) else 1)
-10.70634317730195 -2.1072103131565263 -8.599132864145423 lambda u_a, u_b: (0 if (max([u_a, u_b]) == u_a) else 1)
-10.70634317730195 -2.1072103131565263 -8.599132864145423 lambda u_a, u_b: (1 if (u_b == max([u_b, u_a])) else 0)
-10.70634317730195 -2.1072103131565263 -8.599132864145423 lamb